In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[56]:


import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from chinese_calendar import is_workday, is_holiday
import pickle as pk

def pkload(file_path):
    file = open(file_path, 'rb')
    aaa = pk.load(file)
    file.close()
    return aaa
def pksave(data, file_path):
    file = open(file_path, 'wb')
    pk.dump(data, file)
    file.close()
    return


# In[57]:


time_table = pkload('time_table.pickle')
travel_time_matrix = pkload('travel_time_matrix.pickle')
station_dict = {
    0 : '湘湖',
    1 : '滨康路',
    2 : '西兴',
    3 : '滨和路',
    4 : '江陵路',
    5 : '近江',
    6 : '婺江路',
    7 : '城站',
    8 : '定安路',
    9 : '龙翔桥',
    10 : '凤起路',
    11 : '武林广场',
    12 : '西湖文化广场',
    13 : '打铁关',
    14 : '闸弄口',
    15 : '火车东站',
    16 : '彭埠',
    17 : '七堡',
    18 : '九和路',
    19 : '九堡',
    20 : '客运中心',
    21 : '下沙西',
    22 : '金沙湖',
    23 : '高沙路',
    24 : '文泽路',
    25 : '文海南路',
    26 : '云水',
    27 : '下沙江滨',
    28 : '乔司南',
    29 : '乔司',
    30 : '翁梅',
    31 : '余杭高铁站',
    32 : '南苑',
    33 : '临平',
    34 : '朝阳',
    35 : '曹家桥',
    36 : '潘水',
    37 : '人民路',
    38 : '杭发厂',
    39 : '人民广场',
    40 : '建设一路',
    41 : '建设三路',
    42 : '振宁路',
    43 : '飞虹路',
    44 : '盈丰路',
    45 : '钱江世纪城',
    46 : '钱江路',
    47 : '庆春广场',
    48 : '庆菱路',
    49 : '建国北路',
    50 : '中河北路',
    51 : '凤起路',
    52 : '武林门',
    53 : '沈塘桥',
    54 : '下宁桥',
    55 : '学院路',
    56 : '古翠路',
    57 : '丰潭路',
    58 : '文新',
    59 : '三坝',
    60 : '虾龙圩',
    61 : '三墩',
    62 : '墩祥街',
    63 : '金家渡',
    64 : '白洋',
    65 : '杜甫村',
    66 : '良渚',
    67 : '浦沿',
    68 : '杨家墩',
    69 : '中医药大学',
    70 : '联庄',
    71 : '水澄桥',
    72 : '复兴路',
    73 : '南星桥',
    74 : '甬江路',
    75 : '城星路',
    76 : '市民中心',
    77 : '江锦路',
    78 : '景芳',
    79 : '新塘',
    80 : '新风'
}

station_name_to_id = {v : k for k, v in station_dict.items()}


# In[58]:


def stationID(x):
    new_dict = {v : k for k, v in station_dict.items()}
    return new_dict[x]


# In[59]:


station_name_to_id


# In[60]:


travel_time_matrix


# In[61]:


time_table


# In[67]:


hangzhou_subway = {}
for i in time_table.keys():
    if (i[:3] == '终点站'):
        continue
    hangzhou_subway[i] = {}
    for j in time_table[i].keys():
        if (j[:3] == '终点站'):
            continue
        hangzhou_subway[i][j] = time_table[i][j][0][1][:2]


# In[68]:


hangzhou_subway


# In[69]:


def path_search(start, goal):
    if (start[:3] == '终点站') or (goal[:3] == '终点站'):
        return
    if start == goal:
        return [start]
    explored = set() 
    explored.add(start)
    queue = [[start, ('', 0)]]
    while queue:
        path = queue.pop(0)
        s = path[-2]
        linenum, changetimes = path[-1]
        if s == goal:
            new_path = []
            return path
        for state, action in hangzhou_subway[s].items():
            if state not in explored:
                linechange = changetimes
                explored.add(state)
                if linenum != action:
                    linechange += 1
                path2 = path[:-1] + [action, state, (action, linechange)]
                queue.append(path2)
                queue.sort(key=lambda path:path[-1][-1])
    return []


# In[70]:


path_test = path_search('南星桥', '武林门')


# In[73]:


path_test


# In[72]:


path_test1 = {}
for i in range(int(len(path_test) / 2)):
    path_test1[path_test[2 * i]] = path_test[2 * i + 1]
path_test1


# In[19]:


path_matrix = pkload('path_matrix.pickle')


# In[20]:


path_matrix


# In[74]:


num_sensors = 81
dist_mx = np.zeros((num_sensors, num_sensors), dtype=np.float32)
dist_mx[:] = 0
dist_mx


# In[75]:


def cal_traveltime(path):
    s_flag = 0
    time = 0
    path_list = list(path.keys())
    for i in range(len(path_list)-1):
        A = path_list[s_flag]
        B = path_list[s_flag+1]
        travel_time = travel_time_matrix[A][B]
        time+=travel_time
        s_flag+=1
        print('travel_time:  ',travel_time)
    return time


# In[76]:


for i in station_dict.values():
    startid = stationID(i)
    for j in station_dict.values():
        if i == j:
            endid = startid
            dist_mx[startid][endid]=0
        else:
            endid = stationID(j)
            path = path_matrix[i][j]
            print(' i ', i)
            print(' j ', j)
            time = cal_traveltime(path)
            dist_mx[startid][endid]=time


# In[78]:


path_matrix['新风']['钱江路']


# In[80]:


stationID('钱江路')


# In[45]:


dist_mx


# In[82]:


travel_time_mx = pd.DataFrame(dist_mx)
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)

travel_time_mx


# In[91]:


travel_time_mx.to_csv('W_travel_time.csv',index=False)


# In[87]:


W_t = travel_time_mx.values


# In[89]:


np.max(W_t)


# In[55]:


path_matrix['朝阳']['湘湖']


# In[ ]:




